# Data Processing
This notebook contains the code to merge the wikepedia and and population data. The output of this notebook is the"wp_countries-no_match.txt" file and the "wp_politicians_by_country.csv" file located in the "data_clean" folder.

## Importing Libraries

In [2]:
import json
import pandas as pd

## Loading the Data

In [66]:
# Loading in the wikepedia data, and csvs with politician names, countries, and popultations
with open('../data_intermediate/scores_dict.json', 'r') as file:
    scores_dict = json.load(file)

pol_country = pd.read_csv("../data_raw/politicians_by_country_AUG.2024.csv").drop_duplicates()
pop_country = pd.read_csv("../data_raw/population_by_country_AUG.2024.csv").drop_duplicates()

# Converting the JSON into an easier to use dataframe
scores_df = pd.DataFrame.from_dict(scores_dict, orient = 'index')

# Tracking Missing Matches and Merging the Datasets
Here, we see that most of the missing matches seem legit, except for "Korea, South" and "South Korea", which we therefore adjust to match prior to merging.


In [72]:
# Track and store the missing country matches as a .txt

#Extract unique country names from both datasources
wikepedia_countries_list = set(list(pol_country['country']))
pop_countries_list = set(list(pop_country['Geography']))

missing_countries_wiki = list(set(wikepedia_countries_list) - set(pop_countries_list))
print(missing_countries_wiki)
missing_countries_pop = [country for country in (set(pop_countries_list) - set(wikepedia_countries_list)) if not country.isupper()]
print(missing_countries_pop)

# Changing the South Korea key in the pol_country df
pol_country.loc[pol_country['country'] == 'Korea, South', 'country'] = "Korea (South)"

#Combine the lists
missing_match_countries = missing_countries_wiki + missing_countries_pop

#Write the output to a .txt file
with open('../data_clean/wp_countries-no_match.txt', 'w') as file:
          for country in missing_match_countries:
            file.write(f"{country}\n")

['Guinea-Bissau', 'Korean']
['Georgia', 'Martinique', 'Liechtenstein', 'Kiribati', 'Guam', 'Denmark', 'Canada', 'French Guiana', 'Curacao', 'Western Sahara', 'New Caledonia', 'Australia', 'Nauru', 'Netherlands', 'Dominica', 'Puerto Rico', 'New Zealand', 'Fiji', 'Jamaica', 'Romania', 'San Marino', 'Palau', 'Andorra', 'Ireland', 'Korea (North)', 'French Polynesia', 'China (Hong Kong SAR)', 'Suriname', 'Iceland', 'United Kingdom', 'Mayotte', 'GuineaBissau', 'Brunei', 'China (Macao SAR)', 'Sao Tome and Principe', 'Guadeloupe', 'Philippines', 'Reunion', 'Mexico', 'eSwatini', 'Mauritius', 'United States']


In [73]:
# Merging the scores data with the politician country
merged_df = scores_df.merge(pol_country, left_index = True, right_on = 'name', how='left')

#Assign a region to each country
def label_region(pop_country):
    #Extract names
    geography_list = pop_country['Geography']
    current_region = None

    #Loop through names, assign last region
    for i, name in enumerate(geography_list):
        if name.isupper():
            current_region = name
        pop_country.loc[i, 'Region'] = current_region

    return pop_country

region_pop_country = label_region(pop_country)

#Merging the merged data with country region
full_merged_df = merged_df.merge(region_pop_country, left_on = 'country', right_on = 'Geography', how='inner')

7148


## Processing the Merged Dataset
Upon this analysis, I noticed there were a number of politicians listed multiple times under different countries (40 to be exact). In this case, I have chosen to keep the politician associated with the country that has a larger population. Additionally, we also want to perform post-processing on the dataframe to clean up column names.

In [86]:
# Display politicians with multiple country entries
duplicate_politicians = full_merged_df[full_merged_df['name'].duplicated(keep=False)]

# Visualize the first 10 duplicates
pd.set_option('display.width', 1000)   
print(duplicate_politicians.head(10), '\n')

#Count the total number of politicians with duplicate entries
print("Number of Duplicate Politicians:", len(set(list(duplicate_politicians['name']))))

# Filter the full merged dataset to keep  only the entries corresponding the country with the largest population
max_population_country_row = full_merged_df.groupby('name')['Population'].idxmax()
full_merged_cleaned_df = full_merged_df.loc[max_population_country_row]


#Reformatting the dataframe prior to saving (droppping / renaming columns)
full_merged_cleaned_df = full_merged_cleaned_df.drop(columns=['url', 'Geography'])
full_merged_cleaned_df.rename(columns={"rev_id": "revision_id", 
                                       "Region": "region", 
                                       "name":"article_title", 
                                       "quality_score":"article_quality", 
                                       "Population":"population"}, 
                                       inplace=True)

full_merged_cleaned_df.to_csv('../data_clean/wp_politicians_by_country.csv')

         rev_id quality_score                                               name                                                url  country Geography  Population           Region
151  1248230838          Stub                                        Visar Ymeri          https://en.wikipedia.org/wiki/Visar_Ymeri  Albania   Albania         2.7  SOUTHERN EUROPE
155  1248230838          Stub                                        Visar Ymeri          https://en.wikipedia.org/wiki/Visar_Ymeri   Kosovo    Kosovo         1.7  SOUTHERN EUROPE
166  1250190967             B                                    Oliver Ivanović      https://en.wikipedia.org/wiki/Oliver_Ivanović   Kosovo    Kosovo         1.7  SOUTHERN EUROPE
174  1250189215             C                                        Goran Rakić          https://en.wikipedia.org/wiki/Goran_Rakić   Kosovo    Kosovo         1.7  SOUTHERN EUROPE
419  1249557425             C                                      Hrant Maloyan        https://en.w